In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,119 kB]
Get:15 htt

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-07 04:56:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.46MB/s    in 0.2s    

2020-11-07 04:56:49 (5.46 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_fr.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         FR|      14952|R32VYUWDIB5LKE|0552774294|     362925721|    The God Delusion|               Books|          5|            0|          0|   N|                Y|a propos de ce livre|je conseille fort...| 2013-02-13|
|         FR|      14952|R3CCMP4EV6HAVL|B004GJXQ20|     268067011|A Game of Thrones...|Digital_Ebook_Pur

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date

# Read in the Review dataset as a DataFrame
df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         FR|      14952|R32VYUWDIB5LKE|0552774294|     362925721|    The God Delusion|               Books|          5|            0|          0|   N|                Y|a propos de ce livre|je conseille fort...| 2013-02-13|
|         FR|      14952|R3CCMP4EV6HAVL|B004GJXQ20|     268067011|A Game of Thrones...|Digital_Ebook_Pur

In [7]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_table = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_table.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      26425|             1|
|      29814|             1|
|      20913|             1|
|      28767|             1|
|      20842|             1|
|      31630|             1|
|      31660|             1|
|      17564|             1|
|      24207|             2|
|      32494|             1|
|      14952|             2|
|      24196|             5|
|      28091|             3|
|      28448|             1|
|      31588|             1|
|      28700|             1|
|      32799|             3|
|      28557|             1|
|      21490|             1|
|      28692|             1|
+-----------+--------------+
only showing top 20 rows



In [8]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()

products_table = df.select(["product_id","product_title"]).dropDuplicates()
products_table.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00GIGGS6A|      Huion H610 PRO|
|B00XO12REY|  The Monsanto Years|
|B00170MYIS|Sony NP-FG1 Batte...|
|B000UMXCTY|The Collection (C...|
|B00KCY6576|PIRANHA 2 3D [Blu...|
|B002L6SKIK|          Prometheus|
|B00CW7KK9K|Withings Pulse - ...|
|B00004YLIU|               Kid A|
|B00B4VI8MQ|Epic Adventures: ...|
|B000A3IF8G|The Essential Mic...|
|B00CPZ7DBW|     Jump Birdy Jump|
|B005SODYGA|Made In Germany 1...|
|B008YISMLY| Peter Pan [Blu-ray]|
|0552774294|    The God Delusion|
|B00008QSA5|Super Session + B...|
|B00CJ3V5UK|G.I. Joe 2 : Cons...|
|B0097BY3K6|               Music|
|B00HDEEQ92|Le Hobbit - La dé...|
|B0044JV1K6|Braddock - Missin...|
|B007AR7OGE|First lady of son...|
+----------+--------------------+
only showing top 20 rows



In [9]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

# Load in a sql function to use columns
from pyspark.sql.functions import col

review_id_table = df.select(["review_id","customer_id","product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).dropDuplicates(["review_id"])
# add drop duplicates to avoid errors when uploading to pgAdmin

review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1BSSQ563EWKNN|      28692|0006486118|     586496321| 2013-09-24|
|R20RF32ZCTMRR3|      32494|B00CPZ7DBW|     788907793| 2014-06-16|
|R35CGHFIHGVQQM|      26552|B00D1VKZ3M|      16141818| 2015-01-22|
|R1N4WBVI1KJSPQ|      28091|B007K60MQY|     251269761| 2013-03-16|
|R1RJMTSNCKB9LP|      20842|B00004YLIU|     678427290| 2013-04-15|
|R32VYUWDIB5LKE|      14952|0552774294|     362925721| 2013-02-13|
| RJKULSX2Y5R07|      21490|B00CJ3V5UK|     252503117| 2014-11-07|
| RP2T2TPN9Y5MW|      28091|B00649LT06|     992326428| 2013-03-16|
|R3PS3P7G1ZT57W|      24196|B000A3IF8G|     757843869| 2014-08-21|
| RTGC85CCO9EMU|      26425|B007ZMHBBM|     893659638| 2012-11-22|
| RBCSM1KTYH6FX|      24629|B0031ES8P6|     724566333| 2014-01-13|
|R1OS6CM8GL94ZX|      31333|B00KCY6576|     763997717| 2014-09

In [10]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_table = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine", "verified_purchase"]).dropDuplicates(["review_id"])
# add drop duplicates to avoid errors when uploading to pgAdmin

vine_table.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BSSQ563EWKNN|          5|            0|          0|   N|                Y|
|R20RF32ZCTMRR3|          5|            2|          2|   N|                N|
|R35CGHFIHGVQQM|          2|            2|          2|   N|                Y|
|R1N4WBVI1KJSPQ|          4|            0|          1|   N|                Y|
|R1RJMTSNCKB9LP|          2|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [11]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.ctpgfcjdtxtv.us-east-2.rds.amazonaws.com:5432/dataviz"
config = {"user":"jojobear2020", ##input AWS username
          "password": "Columbiastudent2020#1", ##input AWS password
          "driver":"org.postgresql.Driver"}

In [14]:
# Write review_id_df to table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [15]:
# Write products_df to table in RDS
# about 3 min
products_table.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [20]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_table.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [19]:
# Write vine_df to table in RDS
# 11 minutes
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)